In [1]:
from ris import db2  #library designed for SQL database connection and querying
from ris14 import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from Df_to_PG import* 

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2019-12-06_16:39
E:\RIS\Staff Folders\Samuel\Requests\SH\LT Injury Itx


In [2]:
cdb = db2.PostgresDb('DOTDEVRHPGSQL01', 'CRASHDATA', quiet = True)
gdb = db2.SqlDb('dotgissql01', 'gisgrid', user='GISUSER', db_pass='GISUSER') #Database Connection
msdb = db2.SqlDb('DOT55SQL01', 'DataWarehouse', user='arcgis', db_pass='arcgis') #Database Connection

User name (CRASHDATA):soge
Password (CRASHDATA)········


In [3]:
data =  pd.read_excel('E:\RIS\Staff Folders\Samuel\Requests\SH\LT Injury Itx\Feasible Locations to Review Crashes.xlsx')

In [4]:
x = data.loc[(data['Provide LT Ped & Bike Injuries']=='Yes') | (data['Provide RT Ped & Bike Injuries']=='Yes')]

In [5]:
x = x.rename(columns={"LTTC/RTTC feasible leg(s)" : "LTTC/RTTC feasible legs"})

In [6]:
x = x.rename(columns={"Multilane (Yes/No)" : "Multilane Yes/No"})

In [7]:
df_to_sql(x,'LTRT',cdb)

'Complete'

In [9]:
result = cdb.query("""    
                    select nodeid, count(gid) from public.nysdot_all nys_a
                    join public.nysdot_vehicle nys_v
                    on nys_a.case_num = nys_v.case_num
                    where accd_type_int in (1,2)
                    and masterid::bigint in (SELECT masterid FROM working."LTRT")
                    and nys_v.pre_accd_actn in ('02', '03')
                    and nys_a.case_yr>= 2013 and nys_a.case_yr<=2017
                    group by nodeid
                    """)

In [7]:
result = cdb.query("""    
                    with data as(
                    select distinct nodeid, nys_a.crashid, num_of_inj from public.nysdot_all nys_a
                    join public.nysdot_vehicle nys_v
                    on nys_a.crashid = nys_v.crashid
                    where accd_type_int in (1,2)
                    and masterid::bigint in (SELECT masterid FROM working."LTRT")
                    and nys_v.pre_accd_actn in ('02', '03')
                    and nys_a.case_yr>= 2013 and nys_a.case_yr<=2017
                    and nys_v.veh_typ::int not in (5,6)) 

                    select nodeid, sum(num_of_inj) from data
                    group by nodeid
                    """)

In [10]:
y = pd.DataFrame(result.data, columns = ['nodeid','count'])

In [11]:
y['nodeid'] = map(int, y['nodeid'])
y['count'] = map(int, y['count'])

In [18]:
map(int,list(y['nodeid'].loc[y['count']>=2]))

[78715, 48694, 27317, 17923, 19789, 42344, 35238, 22190, 19819]

In [42]:
cases = cdb.query("""    
                    with data as(
                    select distinct nodeid, nys_a.crashid, nys_a.case_num, nys_a.case_yr, num_of_inj from public.nysdot_all nys_a
                    join public.nysdot_vehicle nys_v
                    on nys_a.crashid = nys_v.crashid
                    where accd_type_int in (1,2)
                    and masterid::bigint in (SELECT masterid FROM working."LTRT")
                    and nys_v.pre_accd_actn in ('02', '03')
                    and nys_a.case_yr>= 2013 and nys_a.case_yr<=2017
                    and nys_v.veh_typ::int not in (5,6)) 

                    select data.nodeid, (ltrt.street1, ltrt.street2) as streets, case_num, case_yr from data
                    join working."LTRT" ltrt
                    on data.nodeid::integer = ltrt.nodeid::integer
                    where data.nodeid::integer in (78715, 48694, 27317, 17923, 19789, 42344, 35238, 22190, 19819)
                    """)

In [43]:
c = pd.DataFrame(cases.data, columns = ['nodeid','streets','case_num', 'case_yr'])

In [44]:
c[['case_num','case_yr']].to_csv('cases_2more.csv', index = False)

In [38]:
c.to_csv('cases.csv', index = False)

In [28]:
x = x.merge(y, how='left', on='nodeid')

In [33]:
x= x.fillna(0)

In [58]:
x.to_csv('LT_RT_injuries.csv')
